## Global imports

Write your imports here so you don't have to write imports below.

In [1]:
!pip install simpy
import simpy as sp
import numpy as np
from typing import List, Tuple
import random
from typing import Callable
import scipy
from scipy.optimize import minimize_scalar

## Problem 1

**Write and discuss the steps to answering the following research question:** We would like to design a [network packet](https://en.wikipedia.org/wiki/Network_packet) routing system. To do so, we will need to define the system and run a simulation so that we can generate data that will be used for optimization of design parameters. Our router will be a bit of a simpler system, so do not worry about any specifics you know about ethernet or telecommunications in general.  The goal of our router is that it will route packets from devices connected to it through to the internet.

We'll parameterize our router so that it can accept $\alpha$ packet routing connections at any time. Assume that packets take $\beta$ time to be routed through the router. Let's say that each device connected to the router is only willing to wait up to $\gamma$ time for the packet to be routed to its destination.  

Let's also say that our system will allow for prioritization on three levels, so that we can make sure those high-priority packets like those for voice and video calls can route through before lower-priority packets like standard file downloads.  

To model our system, let's say that we have $n$ devices on our network connected to the router.  Each device generates $X_v$ videocalling packets, $X_s$ standard priority packets, and $X_l$ low priority packets at an $X_i$ interval time between each packet submitted to the router.  So, say that this device has a total of $X_v + X_s + X_l$ packets.  I would recommend that you shuffle this set of packets.  This device would submit the first of these packets (in any order) to the router at time 0.  The device would then wait $X_i$ before submitting the next packet (which again can be any of those in this device's collection), and again until the total number of packets has been submitted to the router.

Higher priority packets will not preempt lower priority packets, but they will be able to jump the queue (`simpy.PriorityResource`).

Based on the distributions for the parameters below, what is the average number of packets that make it through the router to the internet, for $n$ = [3, 5, 10, 15]?  All time-related parameters below are in milliseconds. Make sure to set the random seed.
- $\alpha$ = 6
- $\beta$ = exponentially-modified normal continuous random variable at location 1 and scale 1.5
- $\gamma$ = 3
- $X_v$ = uniform discrete random variable between [2,10]
- $X_s$ = uniform discrete random variable between [2,10]
- $X_l$ = uniform discrete random variable between [2,10]
- $X_i$ = binomial discrete random variable with n at 10 and p at 0.5

In [2]:
ω= 0
RANDOM_SEED = 15
number_of_devices = 6
gamma = 3
min_beta = 1
max_beta = 1.5

In [3]:
def network(env, number, interval, proc_inc, the_rng):
  for i in range(number):
    p = packet(env, 'Packets %02d' % i, proc_inc, 15.0, the_rng)
    env.process(p)
    t = the_rng.exponential(interval)
    yield env.timeout(t)

In [4]:
def packet(env, name, proc_inc, time_in_bank, the_rng):
  """α time, β time and γ time."""
  packet_list=['Xv','Xs','Xl','Xi']
  router_list=['ethernet','telecommunications']
  random_router = random.choice(router_list)
  random_packet = random.choice(packet_list)
  arrive = env.now
  print(random_packet+' at α %7.2f, %s β' % (arrive, name))
  with proc_inc.request() as req:
    router = the_rng.uniform(min_beta, max_beta)
    results = yield req | env.timeout(router)
    wait = env.now - arrive 
    if req in results:
      print('at time %7.2f, %s γ %6.2f' % (env.now, name, wait))
      tib = the_rng.exponential(time_in_bank)
      yield env.timeout(tib)
      print('at time %7.2f, %s finished their β at router device %random_router' % (env.now, name, random_router))
    else:
        print('at time %7.2f, %s left the line after waiting %6.2f minutes' % (env.now, name, wait))

In [5]:
# Setup and start the simulation
print('SIMULATION:  average number of packets that make it through the router to the internet (time units are minutes)')
the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()
# Start processes and run
proc_inc = sp.Resource(env, capacity=1)
env.process(network(env, number_of_devices, gamma, proc_inc, the_rng))
env.run()

SIMULATION:  average number of packets that make it through the router to the internet (time units are minutes)
Xv at α    0.00, Packets 00 β
at time    0.00, Packets 00 γ   0.00
at time    4.35, Packets 00 finished their β at router device 'ethernet'andom_router
Xi at α    7.50, Packets 01 β
at time    7.50, Packets 01 γ   0.00
Xs at α    7.62, Packets 02 β
at time    8.80, Packets 02 left the line after waiting   1.17 minutes
Xs at α    9.72, Packets 03 β
Xi at α   10.93, Packets 04 β
at time   11.14, Packets 01 finished their β at router device 'ethernet'andom_router
at time   11.14, Packets 03 γ   1.42
at time   12.35, Packets 04 left the line after waiting   1.42 minutes
at time   14.60, Packets 03 finished their β at router device 'ethernet'andom_router
Xi at α   14.70, Packets 05 β
at time   14.70, Packets 05 γ   0.00
at time   40.89, Packets 05 finished their β at router device 'ethernet'andom_router


In [6]:
def network_with_priorities(env, number_of_devices, interval, proc_inc, the_rng):
  for i in range(number_of_devices):
    p = packet_with_priorities(env, 'Packets %02d' % i, proc_inc, 15.0, the_rng)
    priority = np.round(rng.beta(10, 2)).astype(int)
    env.process(p, priority)
    t = the_rng.exponential(interval)
    yield env.timeout(t)
def packet_with_priorities(env, name, proc_inc, time_in_bank, the_rng):
  """α time, β time and γ time."""
  packet_list=['Xv','Xs','Xl','Xi']
  router_list=['ethernet','telecommunications']
  random_router = random.choice(router_list)
  random_packet = random.choice(packet_list)
  arrive = env.now
  print(random_packet+' at α %7.2f, %s β' % (arrive, name))
  with proc_inc.request(priority=packet_priority) as packet_spot:
    router = the_rng.uniform(min_beta, max_beta)
    results = yield packet_spot | env.timeout(router)
    wait = env.now - arrive 
    if req in results:
      print('at time %7.2f, %s γ %6.2f' % (env.now, name, wait))
      tib = the_rng.exponential(time_in_bank)
      yield env.timeout(tib)
      print('at time %7.2f, %s finished their β at router device %random_router' % (env.now, name, random_router))
    else:
        print('at time %7.2f, %s left the line after waiting %6.2f minutes' % (env.now, name, wait))

In [7]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
proc_inc = sp.PriorityResource(the_env, capacity=1)
network_with_priorities(env, number_of_devices, gamma, proc_inc, the_rng)
the_env.run()

## Problem 2

**Write and discuss the steps to answering the following research question:** Create a function that will automatically run the simulation above, introducing a new constraint named $\omega$, which is the length of time the router is willing to dedicate to routing a packet.  Thus, you will need to modify Problem 1 so that $\omega$ is incorporated as another wait time that will interrupt the packet routing.

Then, we would like to see how $\omega$ affects the number of packets that successfully route through. Make sure that the function sets the same random seed inside the function, so that your simulation parameters are consistent across runs. 

Using `scipy.optimize.minimize_scalar()` ([see reference](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html#scipy.optimize.minimize_scalar)), what is the value of $\omega$ (router processing time allocated) that results in the highest number of packets that successfully route through our router design, where $2 \le \omega \le 10$?

In [8]:
def Optimzer():  
  def network(env, number, interval, proc_inc, the_rng):
    for i in range(alpha):
      o = scipy.optimize.minimize_scalar(packet, bounds=opt_bounds, args=(env, 'Packets %02d' % i, proc_inc, 12.0, the_rng))
      env.process(o)
      t = the_rng.exponential(interval)
      yield env.timeout(t)
      #yield env.timeout(t)
  def packet(env, name, proc_inc, time_in_bank, the_rng):
    """α time, β time and γ time."""
    packet_list=['Xv','Xs','Xl','Xi']
    router_list=['ethernet','telecommunications']
    random_router = random.choice(router_list)
    random_packet = random.choice(packet_list)
    arrive = env.now
    print(random_packet+' at α %7.2f, %s β' % (arrive, name))
    
    #print("here,", w)
    with proc_inc.request() as req:
       
      router = the_rng.uniform(min_beta, max_beta)
      results = yield req | env.timeout(router)
      wait = env.now - arrive 
      if req in results:
        print('at time %7.2f, %s γ %6.2f' % (env.now, name, wait))
        tib = the_rng.exponential(time_in_bank)

        yield env.timeout(tib)
        print('at time %7.2f, %s finished their β at router device %random_router' % (env.now, name, random_router))

      else:
        print('at time %7.2f, %s left the line after waiting %6.2f minutes' % (env.now, name, wait))

the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()
# Start processes and run
proc_inc = sp.Resource(env, capacity=1)
env.process(network(env, number_of_devices, gamma, proc_inc, the_rng))
try:
  env.run()
  print("length of time the router is willing to dedicate to routing a packet", number_of_devices)
except:
  print("An exception occurred")

Xv at α    0.00, Packets 00 β
at time    0.00, Packets 00 γ   0.00
at time    4.35, Packets 00 finished their β at router device 'ethernet'andom_router
Xs at α    7.50, Packets 01 β
at time    7.50, Packets 01 γ   0.00
Xl at α    7.62, Packets 02 β
at time    8.80, Packets 02 left the line after waiting   1.17 minutes
Xv at α    9.72, Packets 03 β
Xv at α   10.93, Packets 04 β
at time   11.14, Packets 01 finished their β at router device 'ethernet'andom_router
at time   11.14, Packets 03 γ   1.42
at time   12.35, Packets 04 left the line after waiting   1.42 minutes
at time   14.60, Packets 03 finished their β at router device 'ethernet'andom_router
Xv at α   14.70, Packets 05 β
at time   14.70, Packets 05 γ   0.00
at time   40.89, Packets 05 finished their β at router device 'ethernet'andom_router
length of time the router is willing to dedicate to routing a packet 6
